In [44]:
import pymysql as mdb
from pprint import pprint as pp
import pandas as pd
import numpy as np

In [4]:


con = mdb.connect('localhost', 'root', 'hats', 'gamepricepred') #host, user, password, #database

In [15]:
'''See whats currently in the table'''
with con:
    cur = con.cursor(mdb.cursors.DictCursor)
    ex_str = "SELECT * FROM Games"
    cur.execute(ex_str)
    table_dict = cur.fetchall()
pp(table_dict)

[{u'Appid': 10, u'Id': 1, u'Steamdbjson': 'sdb_10.json'},
 {u'Appid': 20, u'Id': 2, u'Steamdbjson': 'sdb_20.json'},
 {u'Appid': 30, u'Id': 3, u'Steamdbjson': 'sdb_30.json'},
 {u'Appid': 40, u'Id': 4, u'Steamdbjson': 'sdb_40.json'},
 {u'Appid': 50, u'Id': 5, u'Steamdbjson': 'sdb_50.json'},
 {u'Appid': 60, u'Id': 6, u'Steamdbjson': 'sdb_60.json'},
 {u'Appid': 70, u'Id': 7, u'Steamdbjson': 'steamdb_jsons/sdb_70.json'},
 {u'Appid': 80, u'Id': 8, u'Steamdbjson': 'sdb_80.json'},
 {u'Appid': 100, u'Id': 9, u'Steamdbjson': 'sdb_100.json'},
 {u'Appid': 130, u'Id': 10, u'Steamdbjson': 'sdb_130.json'},
 {u'Appid': 220, u'Id': 11, u'Steamdbjson': 'sdb_220.json'},
 {u'Appid': 240, u'Id': 12, u'Steamdbjson': 'sdb_240.json'},
 {u'Appid': 280, u'Id': 13, u'Steamdbjson': 'sdb_280.json'},
 {u'Appid': 300, u'Id': 14, u'Steamdbjson': 'sdb_300.json'},
 {u'Appid': 320, u'Id': 15, u'Steamdbjson': 'sdb_320.json'},
 {u'Appid': 340, u'Id': 16, u'Steamdbjson': 'sdb_340.json'},
 {u'Appid': 360, u'Id': 17, u'Steam

Add columns to table using steam_db_df: 
steam_base_price_simple (includes x and y lists needed to generate recent price graph, store as two lists in list )
steam_sale_price_simple (includes x and y lists needed to generate sale price graph)

for later:
simplest prediction results?
base price

In [23]:
df = pd.read_pickle('df_steam_db.pkl')
df.head()

,appid,fprice,iprice,price_date,min_fprice,min_iprice,min_iprice_var,min_iprice_slope,min_iprice_intercept
0,2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,297750,"[39.99, 29.99, 29.99, 19.99, 19.99]","[39.99, 39.99, 39.99, 39.99, 39.99]","[1.417120656e+12, 1.434047268e+12, 1.434994431...","[39.99, 29.99, 29.99, 19.99, 19.99]","[39.99, 39.99, 39.99, 39.99, 39.99]",0.000000e+00,0,39.99
2,19080,"[9.99, 2.49, 2.49]","[9.99, 9.99, 9.99]","[1.417120656e+12, 1.434049315e+12, 1.434996542...","[9.99, 2.49, 2.49]","[9.99, 9.99, 9.99]",0.000000e+00,0,9.99
3,278850,"[4.99, 2.49, 2.49, 2.49, 2.49, 1.49, 1.49]","[4.99, 4.99, 4.99, 4.99, 4.99, 4.99, 4.99]","[1.417120656e+12, 1.417513376e+12, 1.417545207...","[4.99, 2.49, 2.49, 2.49, 2.49, 1.49, 1.49]","[4.99, 4.99, 4.99, 4.99, 4.99, 4.99, 4.99]",7.888609e-31,0,4.99
4,367260,"[5.39, 5.39, 4.01, 4.01, 4.01, 4.01, 4.01, 4.01]","[5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99]","[1.430513702e+12, 1.431104712e+12, 1.43404599e...","[5.39, 5.39, 4.01, 4.01, 4.01, 4.01, 4.01, 4.01]","[5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99, 5.99]",7.888609e-31,0,5.99


In [54]:
'''
Create Sample JSON Array column
### turns out this doesn't work unless you have mysql ver 5.7 or greater (currently only 5.6 installed)
'''

'\nCreate Sample JSON Array column\n'

In [59]:
''' 
insert current lists into sql data base as blobs. 
Not the best practice, but will suffice for now.
Blobs will have to reconverted using the string "eval" function
'''
def insert_list(db_column_name, df_column_name):
    with con:
        cur = con.cursor()
        for item in df.iterrows():
            ts = '"' + str(item[1][df_column_name]) + '"' #add " around strings for input, 
            if ts == np.nan or ts == 'nan': #check to see if null value, then convert to appropriate null value for SQL
                ts = 'NULL'
            ai = item[1].appid
            ex_str = "UPDATE Games SET %s = %s WHERE Appid = %d" % (db_column_name,ts,ai)
            cur.execute(ex_str)

db_columns = ['SteamDBTimestamp','SteamDbIprice','SteamDbMinIprice','SteamDbMinFprice','SteamDbFprice']
df_columns = ['price_date','iprice','min_iprice','min_fprice','fprice']
in_dict = dict(zip(db_columns,df_columns))

for k,v in in_dict.iteritems():
    insert_list(k,v)
    

